# CPU Keras CNN with TF dataset solution

We import the datast via the [tensorflow dataset API ](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) and provide a CNN solution in Keras that runs in a reasonable ammount of time on the CPU (and very fast on a GPU). The solution uses 1/5th the paramters of the GPU solution that can be found [here](https://www.kaggle.com/gpreda/tensorflow-keras-gpu-for-chinese-mnist-prediction)

In [ ]:
import os
from pathlib import Path

import tensorflow as tf


os.listdir("../input/chinese-mnist")

## Constants

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

IMG_HEIGHT = 32
IMG_WIDTH = 32
CHANNELS = 1

BATCH_SIZE = 32

## Dataset Generation

In [ ]:
data_dir = Path('../input/chinese-mnist/data/data')
list_ds = tf.data.Dataset.list_files(str(data_dir / '*.jpg'), shuffle=False)
image_count = len(list_ds)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=False)

In [ ]:
# check if list_ds is populated
for f in list_ds.take(5):
    print(f.numpy())

### Train val test split

In [ ]:
train_size = int(image_count * 0.75)
val_size = int(image_count * 0.15)
test_size = int(image_count * 0.1)

train_ds = list_ds.take(train_size)
test_ds = list_ds.skip(train_size)
val_ds = test_ds.take(val_size)
test_ds = test_ds.skip(val_size)

print(tf.data.experimental.cardinality(train_ds).numpy())
print(tf.data.experimental.cardinality(val_ds).numpy())
print(tf.data.experimental.cardinality(test_ds).numpy())

Compose dataset from filenames. Generate labels and decode image.

In [ ]:
def get_label(file_path):
    # Get the class
    label_str = tf.strings.split(tf.strings.split(file_path, "_")[3], ".")[0]
    # Start at 0
    label_number = tf.strings.to_number(label_str, out_type=tf.dtypes.int32, name=None) - 1
    return label_number

def decode_img(img):
    # convert the compressed string to a 1D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=CHANNELS)
    # resize the image to the desired size
    return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
for image, label in train_ds.take(1):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

In [ ]:
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)
test_ds = configure_for_performance(test_ds)

## Dataset Visual

In [ ]:
import matplotlib.pyplot as plt

image_batch, label_batch = next(iter(test_ds))

plt.figure(figsize=(10, 10))
for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(image_batch[i,:,:,0].numpy().astype("uint8"))
    label = label_batch[i]
    plt.title(str(label.numpy()))
    plt.axis("off")

## Keras Model Creation

Creating a simple CNN model with a callback for early stopping.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Input

num_classes = 15

model = tf.keras.Sequential([
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS)),
    # Preprocessing
    layers.experimental.preprocessing.Rescaling(1./255),
    
    # Augmentation
    #layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    layers.experimental.preprocessing.RandomRotation(0.05),
    
    # Conv Maxpool model
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    
    # Regularization
    layers.Dropout(0.25),
    
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
  optimizer='adam',
  loss="sparse_categorical_crossentropy",
  metrics=['accuracy'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_filepath = '/tmp/checkpoint'
cb_checkpoint = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)

In [ ]:
history = model.fit(train_ds, epochs=40, validation_data=val_ds, batch_size=32, callbacks=[cb_checkpoint])

In [ ]:
best_model = model
_ = best_model.load_weights(checkpoint_filepath)

## Model evaluation

In [ ]:
best_model.evaluate(test_ds)

In [ ]:
import numpy as np
eval_lists = []
# Predict in batches
for images, labels in test_ds.take(-1):  # only take first element of dataset
    y_pred = best_model.predict(images)
    y_pred_bool = np.argmax(y_pred, axis=1)
    eval_lists.append(list(zip(y_pred_bool, labels.numpy())))

# Place in format we can use
import itertools
eval_list = list(itertools.chain.from_iterable(eval_lists))
eval_t = list(zip(*eval_list))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(*eval_t))
print(confusion_matrix(*eval_t, labels=range(15)))